In [64]:
import pandas as pd
import numpy as np
import datetime
import xgboost

from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [200]:
data = pd.read_json('suppliers.json')

In [201]:
data

,index,ogrn,reliability,reg_date,main_okved,additionalokveds,authorized_capital,workers,address,claimant,defendant,active_debts,completed_debts,status,RNP,facts,company_name
0,0,1072721007721,высокая,26.04.2007,68.20,"[46.43.1, 46.43.3, 46.45.1, 46.49.33, 46.51, 4...",20000.0,8.0,край Хабаровский г Хабаровск ул Льва Толстого ...,17,1,0.0,0,1.0,0.0,"{'good': '24', 'normal': '6', 'bad': '0'}","ООО СК ""ЛУННЫЙ СВЕТ"""
1,1,1097746252548,средняя,27.04.2009,62.01,"[58, 61.1, 61.10.1, 61.10.9, 62.02, 62.09, 63....",20005040.0,123.0,"г Москва ул Житная Д. 10, ЭТАЖ 1, ПОМЕЩ. VI - ...",57,73,0.0,4,1.0,0.0,"{'good': '17', 'normal': '6', 'bad': '8'}","АО ""ОТС"""
2,2,318619600041146,None,06.03.2018,43.31,"[13.92, 13.92.1, 13.92.2, 13.99, 28.23, 33.20,...",NaN,NaN,обл Ростовская,0,0,0.0,0,1.0,0.0,None,ИП АСКАЛЕПОВ С. Н.
3,3,1207700142616,высокая,27.03.2020,18.13,"[18.12, 46.14.1, 46.15, 46.15.4, 46.15.9, 46.1...",10000.0,6.0,"г Москва проезд Нагатинский 1-й Д. 6, СТР. 1, ...",455,0,0.0,0,1.0,0.0,"{'good': '21', 'normal': '4', 'bad': '4'}","ООО ""ИНТЕРПАКС"""
4,4,1027402933040,высокая,30.12.2002,46.49,"[22.22, 22.23, 22.29, 25.99, 43.21, 43.99, 46....",8400.0,53.0,обл Челябинская г Челябинск ул Телеграфная Д. 44,36,2,0.0,1,1.0,0.0,"{'good': '19', 'normal': '8', 'bad': '2'}","ООО ""КАНЦБЮРО"""
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2659,1207,317774600053292,None,06.02.2017,95.22.1,"[33.12, 43.22, 43.29, 46.43.1, 46.44.2, 46.49,...",NaN,NaN,г Москва,0,0,0.0,0,1.0,0.0,None,ИП Иванов Владимир Серафимович
2660,1208,1157746901290,низкая,01.10.2015,46.69.9,"[32.50, 43.21, 47.43, 47.54, 49.31, 49.32, 49....",10000.0,11.0,"г Москва ул Кржижановского Д. 29, К. 5, ЭТАЖ 4...",0,12,1.0,6,1.0,0.0,"{'good': '15', 'normal': '4', 'bad': '1'}","ООО ""ЭНЕРГИЯ"""
2661,1209,318774600577420,None,23.10.2018,62.02,"[41.20, 42.22, 42.99, 45.20, 47.41, 49.32, 53....",NaN,NaN,г Москва,0,0,0.0,0,0.0,0.0,None,ИП Таничева Любовь Геннадьевна
2662,1210,316774600516140,None,29.11.2016,31.01,"[26.11, 26.12, 26.20, 28.23, 28.99, 31.02, 31....",NaN,NaN,г Москва,0,1,0.0,0,1.0,0.0,None,ИП Левченко Григорий Александрович


In [202]:
data.head()

,index,ogrn,reliability,reg_date,main_okved,additionalokveds,authorized_capital,workers,address,claimant,defendant,active_debts,completed_debts,status,RNP,facts,company_name
0,0,1072721007721,высокая,26.04.2007,68.20,"[46.43.1, 46.43.3, 46.45.1, 46.49.33, 46.51, 4...",20000.0,8.0,край Хабаровский г Хабаровск ул Льва Толстого ...,17,1,0.0,0,1.0,0.0,"{'good': '24', 'normal': '6', 'bad': '0'}","ООО СК ""ЛУННЫЙ СВЕТ"""
1,1,1097746252548,средняя,27.04.2009,62.01,"[58, 61.1, 61.10.1, 61.10.9, 62.02, 62.09, 63....",20005040.0,123.0,"г Москва ул Житная Д. 10, ЭТАЖ 1, ПОМЕЩ. VI - ...",57,73,0.0,4,1.0,0.0,"{'good': '17', 'normal': '6', 'bad': '8'}","АО ""ОТС"""
2,2,318619600041146,None,06.03.2018,43.31,"[13.92, 13.92.1, 13.92.2, 13.99, 28.23, 33.20,...",NaN,NaN,обл Ростовская,0,0,0.0,0,1.0,0.0,None,ИП АСКАЛЕПОВ С. Н.
3,3,1207700142616,высокая,27.03.2020,18.13,"[18.12, 46.14.1, 46.15, 46.15.4, 46.15.9, 46.1...",10000.0,6.0,"г Москва проезд Нагатинский 1-й Д. 6, СТР. 1, ...",455,0,0.0,0,1.0,0.0,"{'good': '21', 'normal': '4', 'bad': '4'}","ООО ""ИНТЕРПАКС"""
4,4,1027402933040,высокая,30.12.2002,46.49,"[22.22, 22.23, 22.29, 25.99, 43.21, 43.99, 46....",8400.0,53.0,обл Челябинская г Челябинск ул Телеграфная Д. 44,36,2,0.0,1,1.0,0.0,"{'good': '19', 'normal': '8', 'bad': '2'}","ООО ""КАНЦБЮРО"""


### Классы не сбалансированы ### 

In [203]:
data['reliability'].value_counts()

высокая    1568
низкая      276
средняя      27
Name: reliability, dtype: int64

In [204]:
1568 / 276

5.681159420289855

In [231]:
276/1568

0.1760204081632653

In [205]:
features = ['days_existed', 'authorized_capital', 'workers',
           'claimant', 'defendant', 'active_debts', 
           'completed_debts', 'RNP']

ans = ['reliability']

In [206]:
def count_days(reg_date):
    if reg_date == None:
        return None
    
    try: 
        date = str(reg_date)
        day, month, year = list(map(int, date.split('.')))
        today = datetime.datetime.now()
        bday = datetime.datetime(year, month, day, 11, 59)
        time_diff = today - bday

        return time_diff.days
    except:
        return None

In [207]:
data['days_existed'] = data['reg_date'].apply(count_days)

In [208]:
data.head()

,index,ogrn,reliability,reg_date,main_okved,additionalokveds,authorized_capital,workers,address,claimant,defendant,active_debts,completed_debts,status,RNP,facts,company_name,days_existed
0,0,1072721007721,высокая,26.04.2007,68.20,"[46.43.1, 46.43.3, 46.45.1, 46.49.33, 46.51, 4...",20000.0,8.0,край Хабаровский г Хабаровск ул Льва Толстого ...,17,1,0.0,0,1.0,0.0,"{'good': '24', 'normal': '6', 'bad': '0'}","ООО СК ""ЛУННЫЙ СВЕТ""",6002.0
1,1,1097746252548,средняя,27.04.2009,62.01,"[58, 61.1, 61.10.1, 61.10.9, 62.02, 62.09, 63....",20005040.0,123.0,"г Москва ул Житная Д. 10, ЭТАЖ 1, ПОМЕЩ. VI - ...",57,73,0.0,4,1.0,0.0,"{'good': '17', 'normal': '6', 'bad': '8'}","АО ""ОТС""",5270.0
2,2,318619600041146,None,06.03.2018,43.31,"[13.92, 13.92.1, 13.92.2, 13.99, 28.23, 33.20,...",NaN,NaN,обл Ростовская,0,0,0.0,0,1.0,0.0,None,ИП АСКАЛЕПОВ С. Н.,2035.0
3,3,1207700142616,высокая,27.03.2020,18.13,"[18.12, 46.14.1, 46.15, 46.15.4, 46.15.9, 46.1...",10000.0,6.0,"г Москва проезд Нагатинский 1-й Д. 6, СТР. 1, ...",455,0,0.0,0,1.0,0.0,"{'good': '21', 'normal': '4', 'bad': '4'}","ООО ""ИНТЕРПАКС""",1283.0
4,4,1027402933040,высокая,30.12.2002,46.49,"[22.22, 22.23, 22.29, 25.99, 43.21, 43.99, 46....",8400.0,53.0,обл Челябинская г Челябинск ул Телеграфная Д. 44,36,2,0.0,1,1.0,0.0,"{'good': '19', 'normal': '8', 'bad': '2'}","ООО ""КАНЦБЮРО""",7580.0


In [229]:
len(data)

2664

## Предобработка данных ##

Обрабатываем пропуски + нормализуем данные

In [209]:
X_raw = (data.dropna(subset=ans))[features].to_numpy()
y_raw = (data.dropna(subset=ans))[ans].to_numpy()

In [210]:
def convert2float(array2d, dtp, np_dtp):
    new_arr = np.zeros_like(array2d, dtype=np_dtp)
    
    for i in range(array2d.shape[0]):
        for j in range(array2d.shape[1]):
            try:
                new_arr[i][j] = dtp(array2d[i][j])
            except:
                new_arr[i][j] = np.nan
                
    return new_arr

In [211]:
X = convert2float(X_raw, float, np.float32)
y = convert2float(y_raw, str, '<U10')

In [212]:
y[0:5]

array([['высокая'],
       ['средняя'],
       ['высокая'],
       ['высокая'],
       ['высокая']], dtype='<U10')

In [213]:
imp_mean = SimpleImputer(missing_values=np.nan,
                         strategy='median')
imp_mean.fit(X, y)
X = imp_mean.transform(X)

In [214]:
scaler = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X)

In [215]:
X_train, X_test, y_train, y_test = train_test_split( \
            X, y.squeeze(), test_size=0.20, shuffle=True)

## Логистическая регрессия ##

In [216]:
LogReg = LogisticRegression(class_weight='balanced',
                            penalty='l1',
                            solver='liblinear',
                            verbose=1)
LogReg.fit(X_train, y_train)

[LibLinear]

LogisticRegression(class_weight='balanced', penalty='l1', solver='liblinear',
                   verbose=1)

In [217]:
LogReg.score(X_test, y_test)

0.8346666666666667

In [218]:
preds = LogReg.predict(X_test)
N = len(X_test)

book = {
    'высокая': [0, 0],
    'низкая': [0, 0],
    'средняя': [0, 0], 
} 

for i in range(N):
    if preds[i] == y_test[i]:
        book[y_test[i]][0] += 1
    book[y_test[i]][1] += 1

In [219]:
book, N

({'высокая': [305, 320], 'низкая': [3, 50], 'средняя': [5, 5]}, 375)

## Random Forest ##

In [220]:
forest_classif = RandomForestClassifier()

In [221]:
forest_classif.fit(X_train, y_train)

RandomForestClassifier()

In [222]:
forest_classif.score(X_test, y_test)

0.856

In [223]:
preds = forest_classif.predict(X_test)
N = len(X_test)

book = {
    'высокая': [0, 0],
    'низкая': [0, 0],
    'средняя': [0, 0], 
} 

for i in range(N):
    if preds[i] == y_test[i]:
        book[y_test[i]][0] += 1
    book[y_test[i]][1] += 1

In [224]:
book, N

({'высокая': [309, 320], 'низкая': [7, 50], 'средняя': [5, 5]}, 375)

In [232]:
x_raw = data[features].to_numpy()

In [233]:
x = convert2float(x_raw, float, np.float32)

In [234]:
x = imp_mean.transform(x)

In [235]:
x = scaler.transform(x)

In [236]:
quality = forest_classif.predict_proba(x)

In [240]:
quality

array([[9.96504925e-01, 3.43391255e-03, 6.11620795e-05],
       [2.40000000e-01, 1.70000000e-01, 5.90000000e-01],
       [9.65852965e-01, 3.41470351e-02, 0.00000000e+00],
       ...,
       [9.78252222e-01, 2.17477777e-02, 0.00000000e+00],
       [9.50425939e-01, 4.95740606e-02, 0.00000000e+00],
       [9.13373745e-01, 8.66262554e-02, 0.00000000e+00]])

In [241]:
qual_to_add = quality[:, 0]
qual_to_add

array([0.99650493, 0.24      , 0.96585296, ..., 0.97825222, 0.95042594,
       0.91337374])

In [242]:
data['supplierRating'] = qual_to_add

In [245]:
data.head()

,index,ogrn,reliability,reg_date,main_okved,additionalokveds,authorized_capital,workers,address,claimant,defendant,active_debts,completed_debts,status,RNP,facts,company_name,days_existed,supplierRating
0,0,1072721007721,высокая,26.04.2007,68.20,"[46.43.1, 46.43.3, 46.45.1, 46.49.33, 46.51, 4...",20000.0,8.0,край Хабаровский г Хабаровск ул Льва Толстого ...,17,1,0.0,0,1.0,0.0,"{'good': '24', 'normal': '6', 'bad': '0'}","ООО СК ""ЛУННЫЙ СВЕТ""",6002.0,0.996505
1,1,1097746252548,средняя,27.04.2009,62.01,"[58, 61.1, 61.10.1, 61.10.9, 62.02, 62.09, 63....",20005040.0,123.0,"г Москва ул Житная Д. 10, ЭТАЖ 1, ПОМЕЩ. VI - ...",57,73,0.0,4,1.0,0.0,"{'good': '17', 'normal': '6', 'bad': '8'}","АО ""ОТС""",5270.0,0.240000
2,2,318619600041146,None,06.03.2018,43.31,"[13.92, 13.92.1, 13.92.2, 13.99, 28.23, 33.20,...",NaN,NaN,обл Ростовская,0,0,0.0,0,1.0,0.0,None,ИП АСКАЛЕПОВ С. Н.,2035.0,0.965853
3,3,1207700142616,высокая,27.03.2020,18.13,"[18.12, 46.14.1, 46.15, 46.15.4, 46.15.9, 46.1...",10000.0,6.0,"г Москва проезд Нагатинский 1-й Д. 6, СТР. 1, ...",455,0,0.0,0,1.0,0.0,"{'good': '21', 'normal': '4', 'bad': '4'}","ООО ""ИНТЕРПАКС""",1283.0,0.868658
4,4,1027402933040,высокая,30.12.2002,46.49,"[22.22, 22.23, 22.29, 25.99, 43.21, 43.99, 46....",8400.0,53.0,обл Челябинская г Челябинск ул Телеграфная Д. 44,36,2,0.0,1,1.0,0.0,"{'good': '19', 'normal': '8', 'bad': '2'}","ООО ""КАНЦБЮРО""",7580.0,0.978333


In [246]:
data.to_json('suppliers_with_rating.json')

## XGBoost ##

In [102]:
from numpy import mean
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
import xgboost as xgb
from xgboost import XGBClassifier

In [103]:
X = convert2float(X_raw, float, np.float32)
y = convert2float(y_raw, str, '<U10').squeeze()

In [104]:
y

array(['высокая', 'средняя', 'высокая', ..., 'высокая', 'высокая',
       'низкая'], dtype='<U10')

In [108]:
size = np.count_nonzero(y != 'средняя')
XBoost = np.zeros_like(X[:size])
yBoost = np.zeros_like(y[:size], dtype=int)

cnt = 0
skip = 0
while cnt < len(X):
    if y[cnt] == 'средняя':
        skip += 1
    else:
        XBoost[cnt-skip] = X[cnt]
        if y[cnt] == 'высокая':
            yBoost[cnt-skip] = 1
        elif y[cnt] == 'низкая':
            yBoost[cnt-skip] = 0 
    cnt += 1

In [109]:
yBoost[-5:]

array([1, 1, 1, 1, 0])

In [243]:
model = XGBClassifier(scale_pos_weight=5.68)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)

In [244]:
scores = cross_val_score(model, XBoost, yBoost, 
                         scoring='f1', cv=cv, n_jobs=-1)
scores

array([0.91540785, 0.92675635, 0.92238806, 0.91044776, 0.92261002,
       0.92631579, 0.91540785, 0.91566265, 0.92492492, 0.92030075,
       0.92814371, 0.9246988 , 0.91591592, 0.92145015, 0.9246988 ])